In [66]:
import config
import json
import numpy as np
import pandas as pd
import requests
import warnings
import time
warnings.filterwarnings("ignore")

In [2]:
# Import clean spotify playlist & song data
data = '/Users/florencialeoni/code/omg_i_love_that_song/data/spotify_audio_features_complete.csv'

In [4]:
# Create dataframe
df = pd.read_csv(data)

In [6]:
# Check dataframe first entries
df.head()

,Unnamed: 0,playlist_id,playlist_name,track_uri,track_name,artist_name,album_name,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0,../data/spotify1.json-1,Throwbacks,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,226864.0,4.0,0.0,4.0,0.03110,0.904,0.813,0.006970,0.0471,-7.105,0.1210,0.810,125.461
1,1,../data/spotify1.json-1,Throwbacks,6I9VzXrHxO9rA9A5euc8Ak,Toxic,Britney Spears,In The Zone,198800.0,5.0,0.0,4.0,0.02490,0.774,0.838,0.025000,0.2420,-3.914,0.1140,0.924,143.040
2,2,../data/spotify1.json-1,Throwbacks,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),235933.0,2.0,0.0,4.0,0.00238,0.664,0.758,0.000000,0.0598,-6.583,0.2100,0.701,99.259
3,3,../data/spotify1.json-1,Throwbacks,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justin Timberlake,Justified,267267.0,4.0,0.0,4.0,0.20200,0.891,0.714,0.000234,0.0521,-6.055,0.1400,0.818,100.972
4,4,../data/spotify1.json-1,Throwbacks,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Shaggy,Hot Shot,227600.0,0.0,1.0,4.0,0.05610,0.853,0.606,0.000000,0.3130,-4.596,0.0713,0.654,94.759


In [7]:
# Check dataframe last entries
df.tail()

,Unnamed: 0,playlist_id,playlist_name,track_uri,track_name,artist_name,album_name,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
266381,266381,../data/spotify4.json-1000,other,7c1iI6dwdCpHGYT8DGEfab,Uninvited,Alanis Morissette,The Collection,276200.0,2.0,1.0,4.0,0.02230,0.380,0.538,0.004870,0.0940,-5.493,0.0313,0.193,127.416
266382,266382,../data/spotify4.json-1000,other,4l7YIni9bAiGwWMqK5wvgV,You Learn - 2015 Remastered,Alanis Morissette,Jagged Little Pill (Remastered),239640.0,8.0,1.0,4.0,0.00415,0.401,0.706,0.000000,0.0916,-7.554,0.0404,0.458,168.669
266383,266383,../data/spotify4.json-1000,other,2LVxvGhl2U5p2ql2ujc6vZ,Hands Clean,Alanis Morissette,Under Rug Swept,269400.0,7.0,1.0,4.0,0.00192,0.513,0.820,0.000003,0.5040,-5.428,0.0299,0.520,99.952
266384,266384,../data/spotify4.json-1000,other,2Kgjd9G808FhUDRhS2gidG,Cecilia And The Satellite,Andrew McMahon in the Wilderness,Andrew McMahon In The Wilderness,223880.0,5.0,1.0,4.0,0.01310,0.381,0.844,0.000000,0.3180,-7.123,0.1440,0.239,194.000
266385,266385,../data/spotify4.json-1000,other,1u0l8zWpQeMYStFkc2mLD7,Everywhere,Michelle Branch,The Spirit Room,214827.0,1.0,1.0,4.0,0.01360,0.523,0.828,0.000255,0.1650,-4.696,0.0762,0.476,96.952


In [9]:
# Drop Unnamed column from dataframe
df.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [16]:
# Replace spaces in artist_name and album_name with + signs to pass them as arguments to API calls
f = lambda x: x.replace(" ", "+").strip()
df["artist_name"] = df["artist_name"].map(f)
df["album_name"] = df["album_name"].map(f)

In [18]:
# Replace spaces in track_name with + signs to pass as argument to API calls
df["track_name"] = df["track_name"].map(f)

In [19]:
# Sanity Check
df.head(100)

,playlist_id,playlist_name,track_uri,track_name,artist_name,album_name,duration_ms,key,mode,time_signature,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,genre,summary
0,../data/spotify1.json-1,Throwbacks,0UaMYEvWZi0ZqiDOoHU3YI,Lose+Control+(feat.+Ciara+&+Fat+Man+Scoop),Missy+Elliott,The+Cookbook,226864.0,4.0,0.0,4.0,...,0.904,0.813,0.006970,0.0471,-7.105,0.1210,0.810,125.461,NaN,NaN
1,../data/spotify1.json-1,Throwbacks,6I9VzXrHxO9rA9A5euc8Ak,Toxic,Britney+Spears,In+The+Zone,198800.0,5.0,0.0,4.0,...,0.774,0.838,0.025000,0.2420,-3.914,0.1140,0.924,143.040,NaN,NaN
2,../data/spotify1.json-1,Throwbacks,0WqIKmW4BTrj3eJFmnCKMv,Crazy+In+Love,Beyoncé,Dangerously+In+Love+(Alben+für+die+Ewigkeit),235933.0,2.0,0.0,4.0,...,0.664,0.758,0.000000,0.0598,-6.583,0.2100,0.701,99.259,NaN,NaN
3,../data/spotify1.json-1,Throwbacks,1AWQoqb9bSvzTjaLralEkT,Rock+Your+Body,Justin+Timberlake,Justified,267267.0,4.0,0.0,4.0,...,0.891,0.714,0.000234,0.0521,-6.055,0.1400,0.818,100.972,NaN,NaN
4,../data/spotify1.json-1,Throwbacks,1lzr43nnXAijIGYnCT8M8H,It+Wasn't+Me,Shaggy,Hot+Shot,227600.0,0.0,1.0,4.0,...,0.853,0.606,0.000000,0.3130,-4.596,0.0713,0.654,94.759,NaN,NaN
5,../data/spotify1.json-1,Throwbacks,0XUfyU2QviPAs6bxSpXYG4,Yeah!,Usher,Confessions,250373.0,2.0,1.0,4.0,...,0.881,0.788,0.000000,0.0377,-4.669,0.1680,0.592,104.997,NaN,NaN
6,../data/spotify1.json-1,Throwbacks,68vgtRHr7iZHpzGpon6Jlo,My+Boo,Usher,Confessions,223440.0,5.0,1.0,4.0,...,0.662,0.507,0.000000,0.0465,-8.238,0.1180,0.676,86.412,NaN,NaN
7,../data/spotify1.json-1,Throwbacks,3BxWKCI06eQ5Od8TY2JBeA,Buttons,The+Pussycat+Dolls,PCD,225560.0,2.0,1.0,4.0,...,0.544,0.823,0.000000,0.2680,-4.318,0.3200,0.434,210.750,NaN,NaN
8,../data/spotify1.json-1,Throwbacks,7H6ev70Weq6DdpZyyTmUXk,Say+My+Name,Destiny's+Child,The+Writing's+On+The+Wall,271333.0,5.0,0.0,4.0,...,0.713,0.678,0.000000,0.1490,-3.525,0.1020,0.734,138.009,NaN,NaN
9,../data/spotify1.json-1,Throwbacks,2PpruBYCo4H7WOBJ7Q2EwM,Hey+Ya!+-+Radio+Mix+/+Club+Mix,OutKast,Speakerboxxx/The+Love+Below,235213.0,4.0,0.0,4.0,...,0.728,0.974,0.000532,0.1750,-2.261,0.0665,0.965,79.526,NaN,NaN


In [11]:
# Sanity check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266386 entries, 0 to 266385
Data columns (total 19 columns):
playlist_id         266386 non-null object
playlist_name       266386 non-null object
track_uri           266386 non-null object
track_name          266386 non-null object
artist_name         266386 non-null object
album_name          266386 non-null object
duration_ms         266386 non-null float64
key                 266386 non-null float64
mode                266386 non-null float64
time_signature      266386 non-null float64
acousticness        266386 non-null float64
danceability        266386 non-null float64
energy              266386 non-null float64
instrumentalness    266386 non-null float64
liveness            266386 non-null float64
loudness            266386 non-null float64
speechiness         266386 non-null float64
valence             266386 non-null float64
tempo               266386 non-null float64
dtypes: float64(13), object(6)
memory usage: 38.6+ MB


In [12]:
# Check for null values
nan_rows = df[df.isnull().any(1)]

In [13]:
# Check for null values
print(nan_rows)

Empty DataFrame
Columns: [playlist_id, playlist_name, track_uri, track_name, artist_name, album_name, duration_ms, key, mode, time_signature, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, valence, tempo]
Index: []


In [61]:
# Create genre column
df["genre"] = None

In [62]:
# Create summary column
df["summary"] = None

In [63]:
# Create playcount column
df["playcount"] = None

In [24]:
key = config.last_fm_key

In [67]:
# Iterate row by row making calls to API with track_uri information
for index, row in df.iterrows():
    # Printing index to have a visual check on progress (know which index is being processed)
    print(f"\r{index}", end = "")
    # Check if one of the new columns is empty, if so, start making API calls from that index on
    if pd.isna(row["genre"]):
        # Create artist variable for dynamic URL
        artist = df.at[index, "artist_name"]
        # Create track variable for dynamic URL
        track = df.at[index, "track_name"]
        # Create URL for API calls
        url = "http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key=" + key + "&artist=" + artist + "&track=" + track + "&format=json"
        # Make request to API using get method
        response = requests.get(url)
        # If response is not 200 OK, print error message (to have a visual check on progress)
        time.sleep(0.5)
        while response.status_code != 200:
            print("Response error, waiting.")
            # Wait one second before making next request
            time.sleep(5)
            # Make request and check status again
            response = requests.get(base_url)
        
        # Use json method on the response
        track_info = response.json()
        # Use try except on json object to account for edge cases (like song not found, or empty list of genres)
        try:
            df.at[index, "genre"] = track_info["track"]["toptags"]["tag"][0]["name"]
        except KeyError:
            print("Handle KeyError case.")
            df.at[index, "genre"] = "no_genre"
        except IndexError:
            print("Handle IndexError case.")
            df.at[index, "genre"] = "no_genre"
        try:
            df.at[index, "summary"] = track_info["track"]["wiki"]["summary"]
        except KeyError:
            print("Handle KeyError case.")
            df.at[index, "summary"] = "no_summary"
        except IndexError:
            print("Handle IndexError case.")
            df.at[index, "summary"] = "no_summary"
        try:
            df.at[index, "playcount"] = track_info["track"]["playcount"]
        except KeyError:
            print("Handle KeyError case.")
            df.at[index, "playcount"] = "no_playcount"
        except IndexError:
            print("Handle IndexError case.")
            df.at[index, "playcount"] = "no_playcount"    
            

In [ ]:
# Save the DataFrame created to a csv file
df.to_csv(r'/Users/florencialeoni/code/omg_i_love_that_song/data/songs_last_fm_genre.csv')